In [2]:
import cv2
from imutils.perspective import four_point_transform
from imutils import grab_contours


In [3]:
def find_puzzle(image):
	# convert the image to grayscale and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 3)

    # apply adaptive thresholding and then invert the threshold map

    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    thresh = cv2.bitwise_not(thresh)

    # find contours in the thresholded image and sort them by size in
# descending order
    cnts = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
# initialize a contour that corresponds to the puzzle outline
    puzzleContour = None
# loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # if the approximated contour has four points, then we can
        # assume we have found the outline of the puzzle
        if len(approx) == 4:
            puzzleContour = approx
            break

    # if the puzzle contour is empty then the script could not find
    # the outline of the Sudoku puzzle so raise an error
    if puzzleContour is None:
        raise Exception(("Could not find Sudoku puzzle outline."))

    # apply a four point perspective transform to both the original
    # image and grayscale image to obtain a top-down bird's eye view
    # of the puzzle
    puzzle = four_point_transform(image, puzzleContour.reshape(4, 2))
    warped = four_point_transform(gray, puzzleContour.reshape(4, 2))

    return puzzle, warped

In [5]:
sudoku = cv2.imread('/home/lorenzo/Desktop/Personal_project/digit_dataset/digit_from_grid/bin1.1/sudoku_img/sudoku2.jpg')
puzz, warped  = find_puzzle(sudoku)
   
cv2.imshow("win",puzz)
cv2.waitKey(0)
cv2.destroyAllWindows()